## -- Show the typical duration and number of trips for the 10 most common one-way rentals

In [ ]:
SELECT
  MIN(start_station_name) AS start_station_name,
  MIN(end_station_name) AS end_station_name,
  APPROX_QUANTILES(tripduration, 10)[OFFSET(5)] AS typical_duration,
  COUNT(tripduration) AS num_trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
  start_station_id != end_station_id
GROUP BY
  start_station_id,
  end_station_id
ORDER BY
  num_trips DESC
LIMIT
  10;

## Show top 5 bike's distance

In [ ]:
WITH
  trip_distance AS (
  SELECT
    bikeid,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,s.latitude), ST_GEOGPOINT(e.longitude,e.latitude)) AS distance
  FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`,
    `bigquery-public-data.new_york_citibike.citibike_stations` AS s,
    `bigquery-public-data.new_york_citibike.citibike_stations` AS e
  WHERE
    start_station_name = s.name
    AND end_station_name = e.name )
SELECT
  bikeid,
  SUM(distance)/1000 AS total_distance
FROM
  trip_distance
GROUP BY
  bikeid
ORDER BY
  total_distance DESC
LIMIT
  5;

## Show rainfall (in mm) for all days in 2015 from a weather station in New York with a specified id (the station corresponds to NEW YORK CNTRL PK TWR )

In [ ]:
SELECT
  wx.date,
  wx.value/10.0 AS prcp
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2015` AS wx
WHERE
  id = 'USW00094728'
  AND qflag IS NULL
  AND element = 'PRCP'
ORDER BY
  wx.date;

## Find correlation between rain and bicycle rentals

In [ ]:
WITH bicycle_rentals AS (
  SELECT COUNT(starttime) AS num_trips,
  EXTRACT(DATE from starttime) AS trip_date 
  FROM  `bigquery-public-data.new_york_citibike.citibike_trips`
  GROUP BY trip_date
),

rainy_days AS (
    SELECT date, (MAX(prcp)>5) AS rainy 
    FROM (
      SELECT wx.date AS date,
            IF(wx.element='PRCP', wx.value/10.0, NULL) as prcp
      FROM `bigquery-public-data.ghcn_d.ghcnd_2015` AS wx
      WHERE wx.id = 'USW00094728'
    )
  GROUP BY date
)
SELECT
  ROUND(AVG(bk.num_trips)) AS num_trips,
  wx.rainy
FROM
  bicycle_rentals AS bk
JOIN
  rainy_days AS wx
ON
  wx.date = bk.trip_date
GROUP BY
  wx.rainy;